In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio',1), df['cardio'], random_state=13)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 0.70952944,  0.44281281, -0.42931555, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.49404005,  0.19945049,  0.75000384, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.66531452,  1.17289976, -0.01308517, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-1.73729991,  0.68617512, -0.98428938, ...,  0.        ,
         0.        ,  0.        ],
       [-2.01678689, -1.74744807, -0.42931555, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.941962  ,  1.29458092,  3.17801435, ...,  0.        ,
         0.        ,  1.        ]])

In [6]:
X_train

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
59760,85325,21217,1,168,68.0,120,70,3,3,0,0,0
63217,90274,23151,1,166,85.0,100,70,3,1,0,0,0
30286,43316,21108,2,174,74.0,120,80,1,1,0,0,1
5325,7579,22315,1,154,64.5,130,80,2,2,0,0,0
40086,57303,20589,2,160,79.0,140,80,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
36794,52586,20423,2,180,97.0,130,90,1,3,1,0,1
25324,36155,21759,2,172,84.0,137,91,1,1,1,1,1
65689,93761,15185,2,170,60.0,120,80,1,1,0,0,0
33634,48054,14496,1,150,68.0,120,80,2,1,0,0,1


In [7]:
models = []
models.append(('LogisticRegression', LogisticRegression(random_state=13)))
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('CatBoostClassifier', CatBoostClassifier(silent=True)))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))


In [8]:
total_score = []

for name, model in models:
    model.fit(feature_processing.fit_transform(X_train), y_train)
    y_pred_train  = model.predict(feature_processing.fit_transform(X_train))
    y_pred_test = model.predict(feature_processing.fit_transform(X_test))
    cv_scores = cross_val_score(model, feature_processing.fit_transform(X_train), y_train, cv=5, scoring='roc_auc')
    cv_mean = np.mean(cv_scores)
    cv_std = np.std(cv_scores)
    f1_scores = f1_score(y_test, y_pred_test.round())
    precision = precision_score(y_test, y_pred_test.round())
    recall = recall_score(y_test, y_pred_test.round())
    total_score.append((name, cv_mean, cv_std, f1_scores, precision, recall))

In [9]:
df_total_score = pd.DataFrame(total_score, columns = ['model', 'cv_mean', 'cv_std', 'f1_scores', 'precision', 'recall'])
df_total_score

,model,cv_mean,cv_std,f1_scores,precision,recall
0,LogisticRegression,0.783581,0.001925,0.703311,0.732994,0.675939
1,RandomForestClassifier,0.773939,0.002872,0.704828,0.737250,0.675137
2,CatBoostClassifier,0.802124,0.002224,0.707944,0.764745,0.658997
3,GradientBoostingClassifier,0.803031,0.001731,0.719583,0.754807,0.687500


### Вывод: GradientBoostingClassifier справился лучше всех.

### ROC AUC лучше использовать так она дает понять баланс между истино положительными и ложный положительными показателями
